In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

sber_data = pd.read_csv('data/sber_data.csv')


eco_groups = sber_data.groupby('ecology')['price_doc']





IndexError: Column(s) price_doc already selected